# FAISS for NMT Bitext Tutorial

## Mount Drive & Files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import sys
my_path = '/content/notebooks'
os.symlink('/content/drive/MyDrive/AllforOne/package_collection', my_path)
sys.path.insert(0, my_path)

Mounted at /content/drive


In [ ]:
!nvidia-smi

Tue Mar 28 03:08:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 1. Install FAISS

In [ ]:
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 12.0 MB/s eta 0:00:00


## 2. Install LASER

In [ ]:
!pip install laserembeddings
!python -m laserembeddings download-models

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.8/859.8 KB 25.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 KB 3.1 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.35-py3-none-any.whl size=883989 sha256=249206daebec2ce73546747347418cb85c0f62b54a1cc5a93b70e235633a5e2b
  Stored in directory: /root/.cache/pip/wheels/03/ff/40/07d5f480a834e1a92370122848b068395ddfd581add24b5b18
Successfully built sacremoses

✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


## 3. Get Bitext

In [ ]:
cd /content/drive/MyDrive/AllforOne/Lecture/FAISS

/content/drive/MyDrive/AllforOne/Lecture/FAISS


In [ ]:
cat src.ko

집에 가고 싶다
하지만 이미 집에 있다

In [ ]:
cat tgt.en

I want to go home
I love my home
But I am already in home

### 3-1. Query : Key = Source sentence : Target Document

In [ ]:
import numpy as np
import faiss
from laserembeddings import Laser

# bitext 데이터 로드
with open('tgt.en', 'r', encoding='utf8') as f:
    bitext = f.readlines()

# LASER 모델 로드
laser = Laser()

# 문장 임베딩 생성
embeddings = laser.embed_sentences(bitext, lang='en')

# Faiss 인덱스 생성
d = embeddings.shape[1]
index = faiss.IndexFlatIP(d)
index.add(embeddings)

# 검색할 쿼리 문장
query = '집에 가고 싶다'

# 쿼리 문장의 임베딩 생성
query_emb = laser.embed_sentences([query], lang='ko')[0]

# 검색
D, I = index.search(np.array([query_emb]), k=1)

# 검색된 결과 출력
print("Query: ", query)
print("Key Sentence: ", bitext[I[0][0]])
print("Index: ", I[0][0])

Query:  집에 가고 싶다
Key Sentence:  I want to go home

Index:  0


### 3-2. Query : Key = Source Document : Target Document

In [ ]:
import numpy as np
import faiss
from laserembeddings import Laser

# src.txt와 tgt.txt 데이터 로드
with open('src.ko', 'r', encoding='utf8') as f:
    src_lines = f.readlines()
    
with open('tgt.en', 'r', encoding='utf8') as f:
    tgt_lines = f.readlines()

# LASER 모델 로드
laser = Laser()

# 문장 임베딩 생성
src_embeddings = laser.embed_sentences(src_lines, lang='ko')
tgt_embeddings = laser.embed_sentences(tgt_lines, lang='en')

# Faiss 인덱스 생성
d = src_embeddings.shape[1]
index = faiss.IndexFlatIP(d)
index.add(tgt_embeddings)

# src.txt의 각 문장에서 가장 유사한 문장 검색
for i, src_line in enumerate(src_lines):
    # 쿼리 문장의 임베딩 생성
    query_emb = laser.embed_sentences([src_line], lang='en')[0]
    
    # 검색
    D, I = index.search(np.array([query_emb]), k=1)
    
    # 검색된 결과 출력
    print("Source sentence: ", src_line.strip())
    print("Target sentence: ", tgt_lines[I[0][0]].strip())
    print("Source index: ", i)
    print("Target index: ", I[0][0])
    print()


Source sentence:  집에 가고 싶다
Target sentence:  I want to go home
Source index:  0
Target index:  0

Source sentence:  하지만 이미 집에 있다
Target sentence:  But I am already in home
Source index:  1
Target index:  2

